### Goals of this notebook:
### 1. Preprocesses the data to prepare for modeling
### 2. Modeling using a number of different models


In [53]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

## 1. Preprocessing
We get the data ready to model by dropping some features, creating dummy variables for catagorical features, and scaling the features.

In [54]:
import pickle
os.chdir("C:\Springboard\Github\Capstone2_cust\Intermediate_Data")

In [55]:
# load picked version of X
X = pickle.load(open("X1.pkl", "rb"))
# look at the first 10 rows of this file
X.head(10)

,first_total,Marketing_first,first_items,first_order,server,Vendor,Source,Area_Code,Ship_Zip,lead_sku,weekday,mon,first_tot_lg,first_it_lg
1,145.58,1,2,2019-11-26 21:44:16+00:00,custom,1.0,web,404,30087,other,Tuesday,November,2.163102,0.30103
2,137.55,0,5,2019-11-26 20:52:08+00:00,custom,1.0,web,845,12545,other,Tuesday,November,2.138461,0.69897
3,22.98,1,2,2019-11-26 18:12:04+00:00,custom,1.0,web,262,53402,ROUTEINS10,Tuesday,November,1.361350,0.30103
4,28.00,0,1,2019-08-07 18:14:49+00:00,custom,0.0,web,617,01983,BEM1003,Wednesday,August,1.447158,0.00000
5,12.00,1,1,2019-08-07 18:05:28+00:00,custom,0.0,web,740,43143,other,Wednesday,August,1.079181,0.00000
6,42.00,0,2,2019-08-07 03:45:52+00:00,custom,0.0,web,701,58801,BES1006,Wednesday,August,1.623249,0.30103
7,27.20,1,1,2019-08-06 22:00:54+00:00,custom,0.0,web,754,33026,BEM6001,Tuesday,August,1.434569,0.00000
8,22.00,1,2,2019-08-06 20:22:25+00:00,custom,0.0,web,unknown,01880,BES5001,Tuesday,August,1.342423,0.30103
9,100.00,1,5,2019-08-06 19:59:05+00:00,custom,0.0,web,617,01880,BEM1007,Tuesday,August,2.000000,0.69897
10,36.64,1,2,2019-08-06 19:14:49+00:00,custom,0.0,web,626,92887,BEM1007,Tuesday,August,1.563955,0.30103


That looks pretty good. Let's review what each column means: <br>
- first_total: total $ spend on first order <br>
- Marketing_first: whether they accept marketing on the first order <br>
- first_items: number of items on first order <br>
- first_order: date-time of first order <br>
- server: domain name of the customer email server <br>
- vendor: 0 = first order from company; 1 = first order from outside source <br>
- Source: web or iphone
- Area_Code: area code of order placed
- Ship_Zip: zip code of shipping address
- lead_sku: name of SKU that was lead item on purchase
- weekday: day of week first order was placed
- mon: month that first order was placed
- first_tot_lg: log of first order total
- first_it_lg: log of number of items in first order <br>
<br>
The values for some of these catagorical features need to be converted to numbers

In [56]:
X['Source'].value_counts()

web        33949
1356615     5423
294517       273
457101        67
580111        16
412739         2
Name: Source, dtype: int64

In [57]:
# let's drop first order item log
X.drop('first_it_lg', axis=1, inplace=True)

In [58]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39730 entries, 1 to 39770
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   first_total      39730 non-null  float64            
 1   Marketing_first  39730 non-null  int64              
 2   first_items      39730 non-null  int64              
 3   first_order      39730 non-null  datetime64[ns, UTC]
 4   server           39730 non-null  object             
 5   Vendor           39730 non-null  float64            
 6   Source           39730 non-null  object             
 7   Area_Code        39730 non-null  object             
 8   Ship_Zip         39730 non-null  object             
 9   lead_sku         39730 non-null  object             
 10  weekday          39730 non-null  object             
 11  mon              39730 non-null  object             
 12  first_tot_lg     39730 non-null  float64            
dtypes: datetime64[ns

traditional regression can't handle date-time, so we'll drop that

In [59]:
# drop date-time
X.drop('first_order', axis=1, inplace=True)

features that need dummy variables:
- server
- source
- Area_Code
- Ship_Zip
- lead_sku
- weekday
- mon <br>
<br>
Let's look at the size of these

In [60]:
X['server'].nunique()

26

In [61]:
X['Source'].nunique()

6

In [62]:
X['Area_Code'].nunique()

373

This is too man catagorical variables for this feature. I wonder if there are high concentrations of purchases in some zip codes that we could account for

In [63]:
X['Area_Code'].value_counts().head(20)

unknown    21387
949          273
714          204
720          161
760          155
310          148
214          144
512          144
817          139
801          135
757          132
917          131
208          131
909          130
503          130
704          129
916          129
360          128
619          127
480          121
Name: Area_Code, dtype: int64

There are 40k customers, so the largest area code makes up 0.68% of total orders. I think this feature is too small and would add unnecessary dimensions. We will drop.

In [64]:
X.drop('Area_Code', axis=1, inplace=True)

In [65]:
X['Ship_Zip'].nunique()

15930

That is way too many features. Unless they are very concentrated in a few zip codes, we'll drop it.

In [66]:
X['Ship_Zip'].value_counts().head(20)

92688      48
92692      48
unknown    48
92691      46
28532      41
92694      40
92630      39
92627      38
92656      31
92679      29
92677      27
92675      25
92672      24
92626      23
92629      22
80013      22
92592      22
92660      21
79936      20
93551      20
Name: Ship_Zip, dtype: int64

In [67]:
# drop that feature
X.drop('Ship_Zip', axis=1, inplace=True)

In [68]:
X['lead_sku'].nunique()

26

We know how many variables for days of the week and month there are. This means we should see: <br>
- server: 26
- source: 6
- lead_sku: 26
- weekday: 7
- mon: 12 <br>
- total: 77 more freatures <br>
<br>
That seems reasonable

In [69]:
# dummy for server ##
dfs = X['server']
dummy_server = pd.get_dummies(dfs)
X = pd.concat([X.drop('server', axis=1), dummy_server], axis=1)

In [70]:
# Dummy for Source
dfs = X['Source']
dummy_source = pd.get_dummies(dfs)
X = pd.concat([X.drop('Source', axis=1), dummy_source], axis=1)

In [71]:
# Dummy for lead_sku
dfs = X['lead_sku']
dummy_source = pd.get_dummies(dfs)
X = pd.concat([X.drop('lead_sku', axis=1), dummy_source], axis=1)

In [72]:
# Dummy for weekday
dfs = X['weekday']
dummy_source = pd.get_dummies(dfs)
X = pd.concat([X.drop('weekday', axis=1), dummy_source], axis=1)

In [73]:
# Dummy for weekday
dfs = X['mon']
dummy_source = pd.get_dummies(dfs)
X = pd.concat([X.drop('mon', axis=1), dummy_source], axis=1)
X

,first_total,Marketing_first,first_items,Vendor,first_tot_lg,aim.com,aol.com,att.net,bellsouth.net,charter.net,...,December,February,January,July,June,March,May,November,October,September
1,145.58,1,2,1.0,2.163102,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,137.55,0,5,1.0,2.138461,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,22.98,1,2,1.0,1.361350,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28.00,0,1,0.0,1.447158,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,12.00,1,1,0.0,1.079181,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39766,27.98,0,2,1.0,1.446848,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
39767,59.97,0,2,1.0,1.777934,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
39768,54.97,1,2,1.0,1.740126,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
39769,90.68,0,3,1.0,1.957512,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


That looks good. Let's load Y.

In [74]:
# load picked version of X
y = pickle.load(open("Y1.pkl", "rb"))
# look at the first 10 rows of this file
y.head(10)

1     2.163102
2     2.138461
3     1.361350
4     1.447158
5     1.079181
6     1.623249
7     1.434569
8     1.342423
9     2.000000
10    1.563955
Name: life_lg, dtype: float64

In [75]:
# let's scale the X values
# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)
# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X)

In [76]:
# let's split the training and test sets
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# let's do the split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=1)

In [77]:
# let's quickly check that the Y means to make sure the data sets are similiar
y_train.mean(), y_test.mean()

(1.8158008483225205, 1.8162965793223824)

Those look reasonably close. <br>
<br>
That is the end of preprocessing. Time to model. 

## 2. Modeling
We will try a few different models: <br>
- Linear Regression
- Random Forest Regressor
- Dummy Model (assume every customer is average)

#### Model 1: Linear Regression

In [78]:
#first model from X-train
from sklearn import linear_model
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [79]:
# predicted results from the first model on the X_test values
y_pred = model.predict(X_test)
# Mean Absolute Error
m1_MAE = mean_absolute_error(y_test, y_pred)
m1_MAE

50444183616.4365

In [80]:
# Mean Squared Error
m1_MSE = mean_squared_error(y_test, y_pred)
m1_MSE

1.0109758020001125e+25

These values seem extremely high. I want to look at some of the values in the model really quick.

In [81]:
print(lm.intercept_)

10088836725.066


That seems way off with linear regression. Let's see if Random Forest Regressor looks any better

#### 2. Random Forest Regressor

In [82]:
from sklearn.ensemble import RandomForestRegressor

In [83]:
# using a minimum sample of 40 or 1/1000th of the total number of customers
regr = RandomForestRegressor(min_samples_leaf=40, random_state=33)
model_rf = regr.fit(X_train,y_train)
y_pred_rf = regr.predict(X_test)

In [84]:
# Mean Absolute Error
m2_MAE = mean_absolute_error(y_test, y_pred_rf)
m2_MAE

0.18474874822669013

In [85]:
# Mean Squared Error
m2_MSE = mean_squared_error(y_test, y_pred_rf)
m2_MSE

0.06807649032099493

In [87]:
# root mean squared error
m2_RMSE = m2_MSE**0.5
m2_RMSE

0.2609147184828693

That is so much better! <br>
Let's look at the importance of each feature

In [88]:
coff_m2 = pd.DataFrame(model_rf.feature_importances_, X.columns, columns=['Importance'])
coff_m2.sort_values('Importance', ascending=False).head(20)

,Importance
first_tot_lg,0.510678
first_total,0.430523
November,0.011758
1356615,0.007362
first_items,0.003863
gmail.com,0.003772
December,0.002964
Marketing_first,0.002945
January,0.002182
July,0.002089


These are good see. <br>
We may want to use these values to eliminate some variable from the regression model, but first let's compare the RMSE and AMSE to a dummy model that just assumes very customer is average 

#### 3. Dummy Model (average)

In [89]:
# assume the model is the average of the training data (we know this value)
y_pred_dm = np.full(y_test.size, y_train.mean())
# Mean Absolute Error
m3_MAE = mean_absolute_error(y_test, y_pred_dm)
m3_MAE

0.29065108266816125

In [90]:
# Mean Squared Error
m3_MSE = mean_squared_error(y_test, y_pred_dm)
m3_MSE

0.14451655741084168

In [91]:
# root mean squared error
m3_RMSE = m3_MSE**0.5
m3_RMSE

0.38015333407829227

These are worse than the Random Forest Regressor, so that's a good thing

In [92]:
# difference in root mean squared error between the two models
diff = m3_RMSE - m2_RMSE
diff

0.11923861559542298

In [93]:
# lifetime dollar value of that difference
10**diff

1.3159476582939336

That is honestly not that impressive, but is still better than the dummy model.